**[참고] Open AI API Key 발급절차**  
1.   OpenAI developer platform 접속  
https://platform.openai.com/  
2.   로그인 (계정 없을시 회원가입, 유료결제 후 API 이용가능)  
3.   상단에서 "Dashboard" Click  
4.   좌측에서 "API keys" Click  
5.   "Create new secret key" 클릭하여 API key 발급

*   주의: 보안상 처음 생성 시 한 번만 전체 키가 보임, 다시는 원본을 볼 수 없음


In [1]:
# ! : Jupyter Notebook, Colab, IPython 환경에서 "터미널(쉘)" 명령어로 실행하라는 의미
# pip install : "패키지를 PyPI(Python Package Index, Python 공식 패키지 저장소)에서 다운로드해서 현재 환경에 설치"
# LangChain은 LLM개발을 위한 범용 프레임워크 : 여러 LLM·DB·툴과 연결 가능
!pip install langchain

# OpenAI API를 쓰려면 별도로 langchain-openai를 설치필요
# 코어(langchain)와 분리해서 제공하는 정책
# : Why? 모든 통합 모듈(OpenAI, anthropic 등)을 한꺼번에 설치하면 패키지 용량이 커지고 의존성 충돌
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.9 MB/s eta 0:00:00


In [3]:
# os.environ : 운영체제(OS)에 등록된 환경변수를 파이썬에서 읽거나 쓸 수 있게 해주는 인터페이스
# OpenAI API KEY는 코드 안에 하드코딩하지 않고 환경변수로 저장해두고 읽어오는 방식이 보안상 안전
import os
os.environ["OPENAI_API_KEY"] = ''


## 동기식(Sync) 호출 예시

In [23]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage


# LLM 객체생성
llm = ChatOpenAI(
    model="gpt-4o",          # LLM 모델명
    top_p=0.7,               # 확률 분포 Cut-off
    temperature=1.4,         # 창의성
    max_tokens=500,          # 응답 길이 제한
    # ,api_key=''            # 환경변수 OPENAI_API_KEY를 기본값으로 읽어옴
)

# 대화 메시지 구성
messages = [
    SystemMessage(content="You are a helpful assistant that answers in Korean."),
    HumanMessage(content="생성형 AI가 무엇인지 쉽게 설명해줘.")
]

# 응답 생성
# invoke() : LangChain에서 모델을 한 번 호출할 때 사용하는 메서드 - 동기식(Sync)으로 입력 → 응답을 바로 반환
# ainvoke() : 비동기(Asynchronous) 호출 - 작업을 시작해두고, 끝나길 기다리는 동안 다른 작업을 병행
response = llm.invoke(messages)
print(response.content)


생성형 AI는 주어진 입력을 바탕으로 새로운 콘텐츠를 생성할 수 있는 인공지능 기술을 말합니다. 예를 들어, 텍스트, 이미지, 음악 등을 생성할 수 있는 AI 모델이 이에 해당합니다. 이런 AI는 대량의 데이터를 학습하여 특정 스타일이나 형식을 이해하고, 이를 바탕으로 사람처럼 새로운 결과물을 만들어냅니다. 대표적인 예로는 텍스트 생성 AI인 GPT-3, 이미지 생성 AI인 DALL-E 등이 있습니다. 이러한 기술은 다양한 분야에서 창의적인 작업을 돕거나 자동화하는 데 활용됩니다.


## 스트리밍 호출 예시

In [21]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage


# LLM 객체생성
llm = ChatOpenAI(
    model="gpt-4o",          # LLM 모델명
    top_p=0.7,               # 확률 분포 Cut-off
    temperature=1.4,         # 창의성
    max_tokens=500,          # 응답 길이 제한
    streaming=True           # 스트리밍 활성화
)

# 대화 메시지 구성
messages = [
    SystemMessage(content="You are a helpful assistant that answers in Korean."),
    HumanMessage(content="생성형 AI가 무엇인지 설명해줘.")
]

# 스트리밍 응답 생성
# flush=True: 버퍼링 없이 즉시 출력, 토큰이 들어오는 즉시 바로바로 화면에 출력
#   -> print()는 버퍼에 쌓아뒀다가 한 번에 출력할 수 있음
# end="" : 줄바꿈(\n)없이 바로 이어붙임
for chunk in llm.stream(messages):
    print(chunk.content, end="", flush=True)  # 토큰 단위 출력


# Tip: 출력응답 편하게 보기
#line = ""
#for chunk in llm.stream(messages):
#    line += chunk.content
#    print(chunk.content, end="", flush=True)
#    if len(line) > 100:  # 100자마다 줄바꿈
#        print()
#        line = ""

생성형 AI는 주어진 입력에 기반하여 새로운 콘텐츠를 생성하는 인공지능 기술을 말합니다. 이러한 AI 모델은 텍스트, 이미지, 음악, 코드 등 다양한 형식의 데이터를 생성할 수 있습니다. 생성형 AI는 주로 대량의 데이터를 학습하여 패턴을 인식하고 이를 바탕으로 새로운 데이터를 만들어냅니다.

예를 들어, 텍스트 생성 AI는 주어진 문장의 맥락을 이해하고 그에 맞는 새로운 문장을 작성할 수 있습니다. 이미지 생성 AI는 특정 스타일이나 주제를 기반으로 새로운 이미지를 만들 수 있습니다. 대표적인 생성형 AI 모델로는 OpenAI의 GPT(Generative Pre-trained Transformer) 시리즈와 같은 언어 모델이 있습니다. 이러한 기술은 콘텐츠 생성, 번역, 예술 작품 창작 등 다양한 분야에서 활용되고 있습니다.